# Bringing it Together

Now that we've created a general purpose recommendation engine with Intelligent Recommendations and a ranking and prioritization system with Azure Personalizer, you are ready to tie it all together to get a customized suggestion.

We will select a customer that is currently doing online grocery shopping and has a few items in their cart already.  We will suggest a "Don't forget" item using the top most ranked item from a set of suggested grocery products.   

# Prerequisites

* Complete all steps in the previous 3 notebooks to setup the services and train your models. 
* Intelligent Recommendations & Azure Personalizer Resources setup and congfigured.


# Notebook Setup & Configuration

In [ ]:
import json
import matplotlib.pyplot as plt
import random 
import requests
import time
import uuid
import datetime
import os
import base64
import pandas as pd

## Install Client Library

You only need to run the following cell once to install the client library, you can uncomment it and run from the notebook, or run from a terminal.  

In [ ]:
# Uncomment and run the following line if you haven't already installed the client library.
#pip install azure-cognitiveservices-personalizer

In [ ]:
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankableAction, RewardRequest, RankRequest
from msrest.authentication import CognitiveServicesCredentials

## Connect to Personalizer Service

In [ ]:
# Replace 'personalization_base_url' and 'resource_key' with your valid endpoint values.  You can view these from the Personalizer Resource Azure Portal Page.
personalization_base_url = "Your_Personalizer_url"  
resource_key = "Your_Personalizer_ResourceKey"

In [ ]:
# Instantiate a Personalizer client
client = PersonalizerClient(personalization_base_url, CognitiveServicesCredentials(resource_key))

### API Builder

In [ ]:
# build URLs
personalization_rank_url = personalization_base_url + "personalizer/v1.0/rank"
personalization_reward_url = personalization_base_url + "personalizer/v1.0/events/" #add "{eventId}/reward"
personalization_model_properties_url = personalization_base_url + "personalizer/v1.0/model/properties"
headers = {'Ocp-Apim-Subscription-Key' : resource_key, 'Content-Type': 'application/json'}


### Test Personalizer Service API Call

This should return a "Response [200]" indicating the call was successful, and return the date/time the personalizer model was created and last modified.

In [ ]:
# get model properties
response = requests.get(personalization_model_properties_url, headers = headers, params = None)

print(response)
print(response.json())

## Connect to Intelligent Recommendations Service


In [ ]:
ir_base_uri = 'Your_IR_Service_Endpoint'  # Url from the overview page for the Service Endpoint in the IR account
model = 'Your_Model_Name'  #modeling component in your IR account

client_id = 'Your_Client_ID'  #the application client id you created in previous steps (Notebook 02).
client_secret = 'Your_Client_Secret_Value' #created when you setup Intelligent Recommendations (Notebook 02)

auth_server_url = "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/v2.0/token"
scope = 'c5b731db-1b0a-43f6-bcf6-757667d9cdc6/.default'

In [ ]:
# Encode the client ID and client secret
authorization = base64.b64encode(bytes(client_id + ":" + client_secret, "ISO-8859-1")).decode("ascii")

headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded"
}
body = {
    "grant_type": "client_credentials", "scope" : scope
}

response = requests.post(auth_server_url, data=body, headers=headers)
#print(response.text)


tokens = json.loads(response.text)
token = tokens['access_token']
#print(token)

### API Builder

In [ ]:
def recommendation_uri_builder(base_uri, command, model, seed_item = 0, seed_user = 0, return_count = 100, seed_items_list = '', skip_count = 0):

    #as of July 2022, this is the API path
    ir_path = '/reco/v1.0/'

    
    if command == 'similar':  #FOR FUTURE: algo type for similar are Visual, Textual: '/reco/v1.0/similar/{}?AlgoType=Textual&modeling={}&count={}'.format(seed_item, model, return_count)
        uri = ir_base_uri + ir_path + 'similar/{}?modeling={}&count={}'.format(seed_item, model, return_count)
    elif command == 'popular':
        uri = ir_base_uri + ir_path + 'popular?modeling={}&count={}'.format(model, return_count)
    elif command == 'new':
        uri = ir_base_uri + ir_path + 'new?modeling={}&count={}'.format(model, return_count)
    elif command == 'trending':
        uri = ir_base_uri + ir_path + 'trending?modeling={}&count={}'.format(model, return_count)
    elif command == 'picks':
        #algo types for user picks is only RecentPurchases
        uri = ir_base_uri + ir_path + 'picks?AlgoType=RecentPurchases&modeling={}&userId={}&count={}'.format(model, seed_user, return_count)
    elif command == 'fbt':
        uri = ir_base_uri + ir_path + 'cart/{}?modeling={}&count={}'.format(seed_item, model, return_count)
    elif command == 'cart':
        uri = ir_base_uri + ir_path + 'cart/items?seedItemIds={}&modeling={}&count={}&skipItems={}&userId={}'.format(seed_items_list, model, return_count, skip_count, seed_user)
    else:
        print("No URI could be formed")


    return uri

In [ ]:
def return_recommended_items(uri, token):
    api_call_headers = {'Authorization': 'Bearer ' + token}
    api_call_response = requests.get(uri, headers=api_call_headers)  

    rec_items = pd.json_normalize(json.loads(api_call_response.text)['items'])['id']

    return rec_items

### Test IR Service 

Test the Intelligent Recommendation service by getting a top 10 list of most popular items.

In [ ]:
popular_uri = recommendation_uri_builder(ir_base_uri, 'popular', model, return_count=10)
popular_items = return_recommended_items(popular_uri, token)
print(popular_items)

## Get Product Catalog & Order History

This reference data is used for finding an example basket with items to work from and viewing characteristics of the recommendation results.

In [ ]:
products_df=pd.read_csv('data/products.csv')
print(products_df.shape)
products_df.head()

In [ ]:
orders_df =pd.read_csv('data/orders.csv')
print(orders_df.shape)
orders_df.head(50)

# Setup Test Customer/Scenario

## Select a Customer/Basket

For this example we will use basket 13170, which belongs to Member Number 4524.  

We will assume member 4524 is "Alice" from our file of customers (data/eaters.json).

Alice is currently doing online shopping with a few items in her cart already.  We want to suggest a "Don't forget" item or two while she is shopping that are likely of interest to her.   

We will first use recommendations to get a set of items we could suggest.  For our example, we will do a "cart" style recommendation which considers all the items already in the cart and recommends things that are frequently bought along with the items already in the cart.

Once we have a set of recommendations (we will pick 4), we will set them up as actions, to be ranked by personalizer.  This will allow us to make the best recommendation first, and see if Alice decides to buy it or not.  If she does, we have completed the cycle, if she chooses not to buy it, we will try again.

In [ ]:
basketId = 13170
orders_df[orders_df['basketId'] == basketId]

In [ ]:
member_number = 4524
customer = "Alice"

In [ ]:
# format the results as a comma separated string to pass as parameter to Intelligent Recommendations API.
current_basket_items = orders_df[orders_df['basketId'] == basketId]['itemId']
current_basket_items_list = current_basket_items.to_string(index=False).strip().split('\n')
current_basket_items_string = ','.join(current_basket_items_list)
current_basket_items_string

# Get Recommendations

In [ ]:
current_items = current_basket_items_string 
item_count = 4

cart_picks_uri = recommendation_uri_builder(ir_base_uri, 'cart', model, seed_user=member_number, seed_items_list = current_items, return_count = item_count, skip_count=2)
cart_pick_items = return_recommended_items(cart_picks_uri, token)

In [ ]:
#Recommendations
products_df[products_df['itemId'].isin(cart_pick_items)]

## Add not already picked in basket

At this point we have a set of 4 items we might recommend to Alice.  But which one should we pick?  We are going to use Personalizer to help us decide the best option.

# Get a Personalized Suggestion

## Function: Get Actions

We take the set of recommendations, and format them as "actions" available for the Personalizer Service to rank to prioritize which suggestions to make first, next, etc.  Based on the user's feedback Personalizer can learn from those cycles.

**NOTE:** We are going to randomly add features (type, texture, flavor) to the actions as we don't have more detail on the grocery items available.  However, in a real-world scenario you should try to leverage existing descriptive information about the items.

Products recommended by Intelligent Recommendations for this Market Basket:

![Sample Recommendations](images\item-recommendations.png)

**TIP:** The items returned by your IR service maybe different than what's shown.  You can proceed using the example actions setup in the next step to see the functionality of calling to Personalizer.    If you would like to use your recommendations instead, edit the ids and features data to match the results from your service.  You can explore other categories and features as well, these mimic what we've been using in our examples that Personalizer has "learned" already, but of course it can learn new features too.
 

In [ ]:
def get_actions():
    action1 = RankableAction(id='house keeping products', features=[{"type":"none", "texture":"none", "flavor":"none"}])
    action2 = RankableAction(id='ice cream', features=[{"type":"cold", "texture":"soft", "flavor":"sweet"}])
    action3 = RankableAction(id='other vegetables', features=[{"type":"hot", "texture":"soft", "flavor":"savory"}])
    action4 = RankableAction(id='sausage', features=[{"type":"hot", "texture":"soft", "flavor":"savory"}])

    return [action1, action2, action3, action4]

## Function: Get Context

This function will be used to simulate the context for our example.

Context is information about the user or environment that might impact what recommendation applies at the moment we make a suggestion.  Examples relevant to our grocery scenario could include the time of day, season of year, if it's a holiday or weekend, if the customer is gluten-free, etc.

If your application is a website or mobile app you might lookup the weather or time of day based on a user's location, or access a loyalty application to lookup your customer's known preferences (like organic, gluten-free, etc.), or interactively ask the user a question within the session.

For our use case, we are going to randomly select the "Weather" and the "Flavor" desired for Alice at the time she is doing her grocery shopping.

In [ ]:
def get_context(customer):

    weatheropt = ['Sunny', 'Rainy', 'Snowy']
    weather = weatheropt[random.randint(0,2)]  # we have 3 weather options to randomly pick from

    flavoropt = ['Salty', 'Sweet', 'Plain']
    flavor = flavoropt[random.randint(0,2)]  # we have 3 flavor options to randomly pick from

    return [{'weather': weather, 'flavor': flavor, 'name': customer}]



## Function: Get Ranked Suggestion

Once we have our context/preferences and actiosn available, we request a ranking from personalizer.

**NOTE:** In our example, we are focused more on food items, so we make an adjustment to the call to Personalizer to ignore the action related to housekeeping products.  This could be handled several ways - you could just exclude this item when setting up the actions, or leverage filtering or categories in the recommendations within IR. 

In [ ]:
def get_ranked_suggestion(customer):
    #create a unique id we will use to track each time we make a call to Rank (this can be used to then return "reward" results)
    eventid = str(uuid.uuid4())

    #prepare our actions
    actions = get_actions()

    # get our context
    context = get_context(customer)

    rank_request = RankRequest( actions=actions, context_features=context, excluded_actions=['house keeping products'], event_id=eventid)

    response = client.rank(rank_request=rank_request)

    print("Personalizer service ranked the actions with the probabilities listed below:")

    rankedList = response.ranking
    for ranked in rankedList:
        print(ranked.id, ':',ranked.probability)

    return eventid, context, response.reward_action_id


# Get Suggestion

We are using a set of pre-defined customer choices to verify our results; this could be built based on historical buying history.  Ours is a short history, so not many matches possible.

In [ ]:
# load user context (our eaters' preferences)
users = "data/eaters.json"
with open(users) as handle:
    userpref = json.loads(handle.read())

custprefs = userpref[customer]


Based on what we know about our user, you can determine how well the suggestion performed.  

In [ ]:
# get the suggestion from personalizer
rank_event_id, user_context, suggestion = get_ranked_suggestion(customer)

# get the context values and the associated customer pick
weather = user_context[0]['weather']
flavor = user_context[0]['flavor']
customer_pick = custprefs[weather][flavor]

print('It is {} and {} wants something {}, {} would be the best choice'.format(weather, customer, flavor, customer_pick))
print('Personalizer suggests {}.'.format(suggestion))


Generally, if it's a sunny day and Alice wants something sweet it will pick "ice cream", which is a good choice.  

Based on the limited data we have, other options will likely not make sense.  In a real-world scenario, additional learning cycles with the recommendations would better tune the suggestions.  This could be accomplished by running simulations (see Notebook 03_Personalizer.ipynb) or allowing Personalizer to learn with live interactions - see [Apprentice Mode](https://docs.microsoft.com/en-us/azure/cognitive-services/personalizer/concept-apprentice-mode) for more information.

Keep in mind that sometimes Personalizer will choose to explore and NOT suggest the option with the highest probability returned.  This is a configurable setting, but is important for Personalizer to evolve as user behavior changes in time.  See the documentation for more information on [exploration and exploitation](https://docs.microsoft.com/en-us/azure/cognitive-services/personalizer/concepts-exploration).

# (Optional) Call Reward API

The other part of the learning cycle is to send the results back to Personalizer, indicating the Reward.   In the 03_Personalizer notebook we did this in an iterative cycle to simulate many users.  The code below is similar, just shown here for completeness.    

If the pick seems like a good one, we send a Reward score of 1, and if it's not good we will send a 0.

In [ ]:
reward_score = '0.0'
if (customer_pick == suggestion):
    reward_score = '1.0'

print("Reward Score: {}".format(reward_score))

client.events.reward(event_id=rank_event_id, value=reward_score)
